In [2]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting customer data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch customer data
    print("Fetching customer data...")
    customers = fetch_data_in_bulk('res.partner', ['id', 'name', 'points_balance'], uid)

    if not customers:
        print("No customers fetched. Exiting.")
        return
    print(f"Fetched {len(customers)} customers.")

    # Step 3: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'customer'
        )
        BEGIN
            CREATE TABLE customer (
                id INT PRIMARY KEY,
                customer_name NVARCHAR(255),
                available_points INT
            );
        END
        """)
        conn.commit()

        print("Inserting customer data into SQL Server...")
        batch_data = []

        for customer in customers:
            customer_id = customer.get('id')
            customer_name = customer.get('name')  # The name can be in Arabic or other languages
            available_points = customer.get('points_balance', 0)  # Assuming 'points_balance' is the correct field

            # Add to batch
            batch_data.append((customer_id, customer_name, available_points))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO customer (id, customer_name, available_points)
                VALUES (%d, %s, %d)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO customer (id, customer_name, available_points)
            VALUES (%d, %s, %d)
            """, batch_data)
            conn.commit()

        print("All customer data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting customer data synchronization...
Authenticated successfully. User ID: 112
Fetching customer data...
Error fetching data from Odoo for model res.partner: <Fault 1: 'Traceback (most recent call last):\n  File "/opt/odoo/odoo/addons/base/controllers/rpc.py", line 88, in xmlrpc_2\n    response = self._xmlrpc(service)\n  File "/opt/odoo/odoo/addons/base/controllers/rpc.py", line 68, in _xmlrpc\n    result = dispatch_rpc(service, method, params)\n  File "/opt/odoo/odoo/http.py", line 142, in dispatch_rpc\n    result = dispatch(method, params)\n  File "/opt/odoo/odoo/service/model.py", line 41, in dispatch\n    res = fn(db, uid, *params)\n  File "/opt/odoo/odoo/service/model.py", line 168, in execute_kw\n    return execute(db, uid, obj, method, *args, **kw or {})\n  File "/opt/odoo/odoo/service/model.py", line 94, in wrapper\n    return f(dbname, *args, **kwargs)\n  File "/opt/odoo/odoo/service/model.py", line 175, in execute\n    res = execute_cr(cr, uid, obj, method, *args, **kw)\n